In [21]:
import pandas as pd
import unicodedata
import re
import os

# ---------------------------------------------------
# Helper: convert any string to ASCII-safe version
# - remove accents (é -> e, ñ -> n, …)
# - remove any non-ASCII characters
# ---------------------------------------------------
def clean_to_ascii(s):
    if pd.isna(s):
        return s
    s = str(s)
    # 1) remove accents
    s = ''.join(
        c for c in unicodedata.normalize('NFKD', s)
        if not unicodedata.combining(c)
    )
    # 2) remove all non-ASCII characters (smart quotes, etc.)
    s = re.sub(r'[^\x00-\x7F]+', '', s)
    return s


# ---------------------------------------------------
# 1. Load the cleaned master table from top50_clean.csv
# ---------------------------------------------------
base = pd.read_csv("../data/cleaned/top50_clean.csv")

# Keep only the original 15 columns (no old *_id columns)
base_cols = [
    'Rank', 'Track_Name', 'Artist_Name', 'Genre', 'Beats_Per_Minute',
    'Energy', 'Danceability', 'Loudness_dB', 'Liveness', 'Valence',
    'Length', 'Acousticness', 'Speechiness', 'Popularity', 'Genre_Group'
]
base = base[base_cols].copy()

print("Base columns:", base.columns.tolist())
print("Base shape:", base.shape)

# Handle missing artist / track names (just in case)
base['Artist_Name'] = base['Artist_Name'].fillna('Unknown Artist')
base['Track_Name'] = base['Track_Name'].fillna('Unknown Track')

# ---------------------------------------------------
# 2. Build Artists table 
# ---------------------------------------------------
artists_df = (
    base[['Artist_Name']]
    .sort_values('Artist_Name')
    .reset_index(drop=True)
)
artists_df['artist_id'] = artists_df.index + 1  # 1, 2, ...

print("artists_df shape:", artists_df.shape)
print(artists_df.head())

# ---------------------------------------------------
# 3. Merge artist_id back into the base table
# ---------------------------------------------------
df_merged = base.merge(
    artists_df,
    on='Artist_Name',   # merge on the original artist name
    how='left'
)

missing = df_merged['artist_id'].isna().sum()
print("Rows without artist_id after merge:", missing)
assert missing == 0, "Some Artist_Name values could not be matched to artist_id"

print("df_merged columns:", df_merged.columns.tolist())

# ---------------------------------------------------
# 4. Build Songs table (one row per track)
# ---------------------------------------------------
songs_df = df_merged.copy()
songs_df['song_id'] = range(1, len(songs_df) + 1)  # 1..n

songs_df = songs_df[
    [
        'song_id',
        'Track_Name',
        'artist_id',
        'Genre',
        'Genre_Group',
        'Length',
        'Beats_Per_Minute',
        'Acousticness',
        'Danceability',
        'Loudness_dB',
        'Energy',
        'Liveness',
        'Speechiness',
        'Valence',
    ]
]
songs_df =songs_df.drop_duplicates(subset=['Track_Name', 'artist_id'])

# ---------------------------------------------------
# 5. Build Popularity table (link rank & popularity to each song)
# ---------------------------------------------------
popularity_df = df_merged.copy()
popularity_df['song_id'] = songs_df['song_id']

popularity_mysql = popularity_df[
    ['song_id', 'Rank', 'Popularity']
].rename(
    columns={
        'Rank': 'chart_rank',
        'Popularity': 'popularity_score',
    }
)

# Add primary key music_id = 1..n
popularity_mysql.insert(0, 'music_id', range(1, len(popularity_mysql) + 1))

# Ensure all numeric columns are integer type
for col in ['music_id', 'song_id', 'chart_rank', 'popularity_score']:
    popularity_mysql[col] = popularity_mysql[col].astype(int)

print("popularity_mysql shape:", popularity_mysql.shape)
print(popularity_mysql.head())

# ---------------------------------------------------
# 6. Create ASCII-safe versions for MySQL export
# ---------------------------------------------------

# Artists: rename column and clean to ASCII
artists_mysql = artists_df.rename(
    columns={'Artist_Name': 'artist_name'}
)[['artist_id', 'artist_name']]

artists_mysql['artist_name'] = artists_mysql['artist_name'].apply(clean_to_ascii)

print("artists_mysql shape:", artists_mysql.shape)
print(artists_mysql.head())

# Songs: rename columns to match schema and clean string columns
songs_mysql = songs_df.rename(
    columns={
        'Track_Name': 'track_name',
        'Genre': 'genre',
        'Genre_Group': 'genre_group',
        'Length': 'length_seconds',
        'Beats_Per_Minute': 'beats_per_minute',
        'Acousticness': 'acousticness',
        'Danceability': 'danceability',
        'Loudness_dB': 'loudness_db',
        'Energy': 'energy',
        'Liveness': 'liveness',
        'Speechiness': 'speechiness',
        'Valence': 'valence',
    }
)

text_cols_songs = songs_mysql.select_dtypes(include=['object']).columns
for col in text_cols_songs:
    songs_mysql[col] = songs_mysql[col].apply(clean_to_ascii)

print("songs_mysql shape:", songs_mysql.shape)
print(songs_mysql.head())

# Popularity: only numeric columns, no need for string cleaning
# (already aligned with schema)

# ---------------------------------------------------
# 7. Save all three tables as ASCII CSV files for MySQL Workbench
# ---------------------------------------------------
output_dir = "../data/cleaned"
os.makedirs(output_dir, exist_ok=True)

artists_path = os.path.join(output_dir, "artists_mysql.csv")
songs_path = os.path.join(output_dir, "songs_mysql.csv")
pop_path = os.path.join(output_dir, "popularity_mysql.csv")

artists_mysql.to_csv(artists_path, index=False, encoding="ascii")
songs_mysql.to_csv(songs_path, index=False, encoding="ascii")
popularity_mysql.to_csv(pop_path, index=False, encoding="ascii")

print("Exported CSV files:")
print("  ", artists_path)
print("  ", songs_path)
print("  ", pop_path)

Base columns: ['Rank', 'Track_Name', 'Artist_Name', 'Genre', 'Beats_Per_Minute', 'Energy', 'Danceability', 'Loudness_dB', 'Liveness', 'Valence', 'Length', 'Acousticness', 'Speechiness', 'Popularity', 'Genre_Group']
Base shape: (50, 15)
artists_df shape: (50, 2)
     Artist_Name  artist_id
0      Ali Gatie          1
1       Anuel AA          2
2  Ariana Grande          3
3  Ariana Grande          4
4      Bad Bunny          5
Rows without artist_id after merge: 0
df_merged columns: ['Rank', 'Track_Name', 'Artist_Name', 'Genre', 'Beats_Per_Minute', 'Energy', 'Danceability', 'Loudness_dB', 'Liveness', 'Valence', 'Length', 'Acousticness', 'Speechiness', 'Popularity', 'Genre_Group', 'artist_id']
popularity_mysql shape: (80, 4)
   music_id  song_id  chart_rank  popularity_score
0         1        1           1                79
1         2        2           1                79
2         3        3           2                92
3         4        4           3                85
4         5 